In [23]:
import sys
import math
import random
import subprocess
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) # run at the start of every ipython notebook to use plotly.offline

from functools import reduce
import pandas as pd
df = pd.read_csv('data.csv')



In [24]:
PLOTLY_USERNAME = 'mr-karan'
PLOTLY_KEY = 'tzubazm2ba'

py.sign_in(PLOTLY_USERNAME, PLOTLY_KEY) # your username and api_key go there


In [25]:
def main():
    
    # How many points are in our dataset?
    num_points = 10000
    
    # For each of those points how many dimensions do they have?
    dimensions = 2
    
    # Bounds for the values of those points in each dimension
    lower = -100
    upper = 100
    
    # The K in k-means. How many clusters do we assume exist?
    num_clusters = 4
    
    # When do we say the optimization has 'converged' and stop updating clusters
    opt_cutoff = 0.5
    lat = []
    long = []
    points =[]
    for i in range(df.shape[0]):
        result = df.values[i][0].split("\t")
        formatted=[i.strip(" ") for i in result]
        lat.append(formatted[0])
        long.append(formatted[1])
    pointsds = []
    for i in range(len(lat)):
        pointsds.append([float(lat[i]),float(long[i])])
    for i in pointsds:
        points.append(Point(i))

    # Cluster those data!
    clusters = kmeans(points, num_clusters, opt_cutoff)

    # Print our clusters

    for i,c in enumerate(clusters):
        for p in c.points:
            print (" Cluster: ", i, "\t Point :", p)

    # Display clusters using plotly for 2d data
    # This uses the 'open' command on a URL and may only work on OSX.
    if dimensions == 2 and PLOTLY_USERNAME:
        print ("Plotting points, launching browser ...")
        plotClusters(clusters)

class Point:
    '''
    An point in n dimensional space
    '''
    def __init__(self, coords):
        '''
        coords - A list of values, one per dimension
        '''
        
        self.coords = coords
        self.n = len(coords)
        
    def __repr__(self):
        return str(self.coords)

class Cluster:
    '''
    A set of points and their centroid
    '''
    
    def __init__(self, points):
        '''
        points - A list of point objects
        '''
        
        if len(points) == 0: raise Exception("ILLEGAL: empty cluster")
        # The points that belong to this cluster
        self.points = points
        
        # The dimensionality of the points in this cluster
        self.n = points[0].n
        
        # Assert that all points are of the same dimensionality
        for p in points:
            if p.n != self.n: raise Exception("ILLEGAL: wrong dimensions")
            
        # Set up the initial centroid (this is usually based off one point)
        self.centroid = self.calculateCentroid()
        
    def __repr__(self):
        '''
        String representation of this object
        '''
        return str(self.points)
    
    def update(self, points):
        '''
        Returns the distance between the previous centroid and the new after
        recalculating and storing the new centroid.
        '''
        old_centroid = self.centroid
        self.points = points
        self.centroid = self.calculateCentroid()
        shift = getDistance(old_centroid, self.centroid) 
        return shift
    
    def calculateCentroid(self):
        '''
        Finds a virtual center point for a group of n-dimensional points
        '''
        numPoints = len(self.points)
        # Get a list of all coordinates in this cluster
        coords = [p.coords for p in self.points]
        # Reformat that so all x's are together, all y'z etc.
        unzipped = zip(*coords)
        # Calculate the mean for each dimension
        centroid_coords = [math.fsum(dList)/numPoints for dList in unzipped]
        
        return Point(centroid_coords)

def kmeans(points, k, cutoff):
    
    # Pick out k random points to use as our initial centroids
    initial = random.sample(points, k)
    
    # Create k clusters using those centroids
    clusters = [Cluster([p]) for p in initial]
    
    # Loop through the dataset until the clusters stabilize
    loopCounter = 0
    while True:
        # Create a list of lists to hold the points in each cluster
        lists = [ [] for c in clusters]
        clusterCount = len(clusters)
        
        # Start counting loops
        loopCounter += 1
        # For every point in the dataset ...
        for p in points:
            # Get the distance between that point and the centroid of the first
            # cluster.
            smallest_distance = getDistance(p, clusters[0].centroid)
        
            # Set the cluster this point belongs to
            clusterIndex = 0
        
            # For the remainder of the clusters ...
            for i in range(clusterCount - 1):
                # calculate the distance of that point to each other cluster's
                # centroid.
                distance = getDistance(p, clusters[i+1].centroid)
                # If it's closer to that cluster's centroid update what we
                # think the smallest distance is, and set the point to belong
                # to that cluster
                if distance < smallest_distance:
                    smallest_distance = distance
                    clusterIndex = i+1
            lists[clusterIndex].append(p)
        
        # Set our biggest_shift to zero for this iteration
        biggest_shift = 0.0
        
        # As many times as there are clusters ...
        for i in range(clusterCount):
            # Calculate how far the centroid moved in this iteration
            shift = clusters[i].update(lists[i])
            # Keep track of the largest move from all cluster centroid updates
            biggest_shift = max(biggest_shift, shift)
        
        # If the centroids have stopped moving much, say we're done!
        if biggest_shift < cutoff:
            print ("Converged after %s iterations" % loopCounter)
            break
    return clusters

def getDistance(a, b):
    '''
    Euclidean distance between two n-dimensional points.
    Note: This can be very slow and does not scale well
    '''
    if a.n != b.n:
        raise Exception("ILLEGAL: non comparable points")
    
    ret = reduce(lambda x,y: x + pow((a.coords[y]-b.coords[y]), 2),range(a.n),0.0)
    return math.sqrt(ret)

def plotClusters(data):

    '''
    Use the plotly API to plot data from clusters.

    Gets a plot URL from plotly and then uses subprocess to 'open' that URL
    from the command line. This should open your default web browser.
    '''

    # List of symbols each cluster will be displayed using    
    symbols = ['circle', 'cross', 'triangle-up', 'square']

    # Convert data into plotly format.
    traceList = []
    for i, c in enumerate(data):
        data = []
        for p in c.points:
            data.append(p.coords)
        # Data
        trace = {}
        trace['x'], trace['y'] = zip(*data)
        trace['marker'] = {}
        trace['marker']['symbol'] = symbols[i]
        trace['name'] = "Cluster " + str(i)
        traceList.append(trace)
        # Centroid (A trace of length 1)
        centroid = {}
        centroid['x'] = [c.centroid.coords[0]]
        centroid['y'] = [c.centroid.coords[1]]
        centroid['marker'] = {}
        centroid['marker']['symbol'] = symbols[i]
        centroid['marker']['color'] = 'rgb(200,10,10)'
        centroid['name'] = "Centroid " + str(i)
        traceList.append(centroid)

    # Log in to plotly
    #py = plotly.(username=PLOTLY_USERNAME, key=PLOTLY_KEY)

    # Style the chart
    layout = dict(title = 'Plot',
              xaxis = dict(title = 'X axis'),
              yaxis = dict(title = 'Y axis'),
        plot_bgcolor='lightblue',
              )

    fig = dict(data=traceList, layout=layout)
    iplot(fig)

if __name__ == "__main__": 
    main()

Converged after 6 iterations
 Cluster:  0 	 Point : [26.11330818, -80.09202576]
 Cluster:  0 	 Point : [28.74612042, -81.44248199]
 Cluster:  0 	 Point : [30.19543489, -81.45937347]
 Cluster:  0 	 Point : [29.19432608, -81.04883575]
 Cluster:  0 	 Point : [27.11794142, -80.29223633]
 Cluster:  0 	 Point : [29.52801577, -95.07781982]
 Cluster:  0 	 Point : [32.68078506, -96.5825119]
 Cluster:  0 	 Point : [30.33959457, -86.63752747]
 Cluster:  0 	 Point : [33.82190368, -84.17020416]
 Cluster:  0 	 Point : [34.99295881, -90.0011673]
 Cluster:  0 	 Point : [34.79656495, -92.49948883]
 Cluster:  0 	 Point : [32.49990966, -97.19348145]
 Cluster:  0 	 Point : [35.21810391, -80.8068924]
 Cluster:  0 	 Point : [33.57617203, -81.94897461]
 Cluster:  0 	 Point : [26.12368803, -81.7673111]
 Cluster:  0 	 Point : [36.01992434, -86.87579346]
 Cluster:  0 	 Point : [33.35078941, -84.16101074]
 Cluster:  0 	 Point : [30.47066526, -89.77869415]
 Cluster:  0 	 Point : [32.63532835, -97.10649872]
 Clust